In [44]:
# Importy bibliotek
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier, plot_importance

### Mergujemy to i tamto ...

In [45]:
# Pliki CSV umieszczone w folderze data
uczestnicy = pd.read_csv('data/PPK_Uczestnicy.csv', sep=';')
pracodawcy = pd.read_csv('data/PPK_Pracodawcy.csv', sep=';')

# Mergowanie dwóch csv
df = pd.merge(
    uczestnicy,
    pracodawcy,
    how='left',
    left_on='EMPL_ID',
    right_on='ID'
)


### Wyrzucamy niepotrzebne kolumny 

In [46]:
# Dropowanie niepotrzebnych kolumn
df.drop(columns=['MEMBER_ID',
                 'EMPL_ID',  
                 'LOGICAL_FACTOR_1',
                 'LOGICAL_FACTOR_2',
                 'ID',
                 'PKD_CODE',
                 'PPK_BANK',
                 'NUMERICAL_VALUE'], inplace = True)

# Te kolumny niby jakoś możnaby wykorzystać, ale wątpliwe, że coś by z tego wynikło
df.drop(columns =["CREATED_AT", 
                  "UOZ_START_DATE", 
                  "UOP_SIGN_DATE", 
                  'REGION_CODE',
                  'WORK_START', 
                  'WORK_STOP'], inplace = True)

# no i to trzeba wyrzucic heh
df.drop(columns = ["RESIGN_DATE"], inplace = True)

In [47]:
# Age to int

def to_int(age):
    age = age[:age.find(",")]
    return int(age)

df["AGE"] = df["AGE"].apply(to_int)

In [48]:
# Grouping other nationalities into one category '0'

unique_nat = df['NATIONALITY'].unique()
for val in unique_nat:
    suma = ( df['NATIONALITY'].values == val ).sum()
    if suma < 5000:
        df.loc[df.NATIONALITY == val, 'NATIONALITY'] = 0       

In [49]:
# zamiana danych kategorycznych na dummy variables - tworzymy nową kolumnę na każdy możliwy output kolumny, porównaj poprzednią i następną komórkę
df = pd.get_dummies(df, columns = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'], 
                         prefix = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'])

In [50]:
# duration - czas od zapisania się do teraz
def find_period(start):
    stop = datetime.now().strftime("%Y-%m-%d")
    
    start_year, start_month, start_day = map(int, start.split('-'))
    stop_year, stop_month, stop_day = map(int, stop.split('-'))
    
    days = (stop_year - start_year)*365
    days += (stop_month - start_month)*30 if stop_month > start_month else (start_month - stop_month)*30
    days += stop_day - start_day if stop_day > start_day else start_day - stop_day
    
    return days

df['DURATION'] = df['SIGN_DATE'].apply(find_period) 
df.drop(columns = ['SIGN_DATE'], inplace = True)

In [51]:
# Klasteryzacja wieku

dataset = df[['IS_SUSPENDED', 'AGE']]
# dataset = dataset.groupby('AGE').mean()
# dataset.reset_index(inplace=True)

scaled_dataset = dataset.values

kmeans_kwargs = {
        "init": "random",
        "n_init": 10,
        "max_iter": 300,
        "random_state": 42,
    }
   
# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_dataset)
    sse.append(kmeans.inertia_)

plt.style.use('fivethirtyeight')
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("n_of_clusters")
plt.ylabel("SSE")
plt.show()


kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
n_of_clusters = kl.elbow

kmeans = KMeans(
    init='random',
    n_clusters = n_of_clusters,
    n_init=50,
    max_iter=500,
    random_state=42
    )

kmeans.fit(scaled_dataset)

df['AGE_CL'] = kmeans.predict(df[["IS_SUSPENDED", "AGE"]])

df.drop(columns=["AGE"], inplace=True)

NameError: name 'KMeans' is not defined

### Jak ostatecznie wygląda df

In [ ]:
df

### Podział na dane treningowe i testowe

In [ ]:
# test/train split
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:]], 
                                                    df["IS_SUSPENDED"], 
                                                    stratify=df["IS_SUSPENDED"], 
                                                    test_size=0.10, 
                                                    random_state=56)

### Gradient Boosting

In [ ]:
# klasy są niezbalansowane, więc trzeba przeskalować wagi pozytywnej klasy

pos = df["IS_SUSPENDED"].sum()/len(df)
scalar = (1-pos)/pos

In [ ]:
XGB = XGBClassifier(max_depth=5, 
                  learning_rate=0.1, 
                  objective= 'binary:logistic',
                  n_jobs=-1,
                  use_label_encoder = False,
                  scale_pos_weight = scalar,
                  verbosity=0)
                                
    
xgb_model = XGB.fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(xgb_model.score(X_test[X_train.columns], y_test)))

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_importance(xgb_model, ax=ax)

In [ ]:
# test 

df_test = pd.read_csv("data/PPK_Uczestnicy_TEST.csv", sep=';')
# Mergowanie dwóch csv
df_test = pd.merge(
    df_test,
    pracodawcy,
    how='left',
    left_on='EMPL_ID',
    right_on='ID'
)

# Dropowanie niepotrzebnych kolumn
df_test.drop(columns=['EMPL_ID',  
                 'LOGICAL_FACTOR_1',
                 'LOGICAL_FACTOR_2',
                 'ID',
                 'PKD_CODE',
                 'PPK_BANK',
                 'NUMERICAL_VALUE'], inplace = True)

# Te kolumny niby jakoś możnaby wykorzystać, ale wątpliwe, że coś by z tego wynikło
df_test.drop(columns =["CREATED_AT", 
                  "UOZ_START_DATE", 
                  "UOP_SIGN_DATE", 
                  'REGION_CODE',
                  'WORK_START', 
                  'WORK_STOP'], inplace = True)

# no i to trzeba wyrzucic heh
df_test.drop(columns = ["RESIGN_DATE"], inplace = True)

# Age to int

def to_int(age):
    age = age[:age.find(",")]
    return int(age)

df_test["AGE"] = df_test["AGE"].apply(to_int)

# Grouping other nationalities into one category '0'

unique_nat = df_test['NATIONALITY'].unique()
for val in unique_nat:
    suma = ( df_test['NATIONALITY'].values == val ).sum()
    if suma < 5000:
        df_test.loc[df_test.NATIONALITY == val, 'NATIONALITY'] = 0   
        
# zamiana danych kategorycznych na dummy variables - tworzymy nową kolumnę na każdy możliwy output kolumny, porównaj poprzednią i następną komórkę
df_test = pd.get_dummies(df_test, columns = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'], 
                         prefix = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'])

# duration - czas od zapisania się do teraz
def find_period(start):
    stop = datetime.now().strftime("%Y-%m-%d")
    
    start_year, start_month, start_day = map(int, start.split('-'))
    stop_year, stop_month, stop_day = map(int, stop.split('-'))
    
    days = (stop_year - start_year)*365
    days += (stop_month - start_month)*30 if stop_month > start_month else (start_month - stop_month)*30
    days += stop_day - start_day if stop_day > start_day else start_day - stop_day
    
    return days

df_test['DURATION'] = df_test['SIGN_DATE'].apply(find_period) 
df_test.drop(columns = ['SIGN_DATE'], inplace = True)

In [ ]:
df_test.head()

In [ ]:
y_pred = xgb_model.predict( df_test[df_test.columns[1:]])

In [ ]:
df_pred = pd.DataFrame(data = {"MEMBER_ID": df_test['MEMBER_ID'],"IS_SUSPENDED": y_pred})

In [ ]:
df_pred.shape

In [ ]:
df_pred.to_csv('submission.csv', index=False, sep=';')